In [15]:
import numpy as np
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier

In [2]:
X, y = fetch_openml('mnist_784', version=1, cache=True, return_X_y=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [3]:
train_x, val_x, test_x = X_scaled[:10000], X_scaled[50000:60000], X_scaled[60000:] # total is 70000
train_y, val_y, test_y = y[:10000], y[50000:60000], y[60000:]

In [21]:
svc = SVC(C=1.0, probability=True)
svc.fit(train_x, train_y)
print(svc.score(train_x, train_y), end=" ")
print(svc.score(val_x,val_y))

0.9831 0.9467


In [5]:
rfc = RandomForestClassifier(n_estimators=200, max_leaf_nodes=5000, n_jobs=-1)
rfc.fit(train_x, train_y)
print(rfc.score(train_x, train_y), end=" ")
print(rfc.score(val_x,val_y))

1.0 0.9544


In [6]:
etc = ExtraTreesClassifier(n_estimators=200, max_leaf_nodes=5000, n_jobs=-1)
etc.fit(train_x, train_y)
print(etc.score(train_x, train_y), end=" ")
print(etc.score(val_x,val_y))

1.0 0.9596


In [7]:
vclf = VotingClassifier(estimators=[('svc', svc), ('rfc', rfc), ('etc', etc)], voting='soft')
vclf.fit(train_x, train_y)
print(vclf.score(train_x, train_y), end=" ")
print(vclf.score(val_x,val_y))

1.0 0.9622


In [8]:
print(vclf.score(test_x,test_y))

0.9561


In [18]:
def predict_first_layer(x):
    part1=svc.predict_proba(x)
    part2=rfc.predict_proba(x)
    part3=etc.predict_proba(x)
    return np.concatenate((part1, part2, part3), axis=1)

In [22]:
meta_svc = SVC(C=1.0)
meta_svc.fit(predict_first_layer(val_x), val_y)
print(meta_svc.score(concat_x, val_y), end=" ")
print(meta_svc.score(predict_first_layer(test_x), test_y))

0.9682 0.9579
